# Homework #5

## Importing the relevant libraries
In this homework, I will mainly utilise the libsvm package, from the svmutil.

In [2]:
from svmutil import *
import pandas as pd
import numpy as np

## Importing the data
We will use the numpy.asmatrix and pandas.read_csv methods to convert the csv files datas into numpy matrices/arrays for ease of handling.

In [3]:
train_data = np.asarray(pd.read_csv('X_train.csv',header=None))
train_label = np.asarray(pd.read_csv('T_train.csv',header=None))
test_data = np.asarray(pd.read_csv('X_test.csv',header=None))
test_label = np.asarray(pd.read_csv('T_test.csv',header=None))

## Building the models and performing the predictions
As mentioned, The implementation is based on libsvm. we feed svm_train() with the training data and label to build the model m, with parameter "-t" indicating the kernel. 0 is for Linear, 1 is for Polynomial, and 2 is for RBF.

The model is then evaluated with svm_predict by performing the prediction on the test data.

### Linear model

In [42]:
m = svm_train(train_label.flatten(), train_data, '-t 0')
lin_result = svm_predict(test_label.flatten(),test_data,m)

Accuracy = 95.08% (2377/2500) (classification)


### Polynomial model

In [43]:
m = svm_train(train_label.flatten(), train_data, '-t 1')
poly_result = svm_predict(test_label.flatten(),test_data,m)

Accuracy = 34.68% (867/2500) (classification)


### RBF model

In [44]:
m = svm_train(train_label.flatten(), train_data, '-t 2')
rbf_result = svm_predict(test_label.flatten(),test_data,m)

Accuracy = 95.32% (2383/2500) (classification)


As seen from above, linear and rbf kernel both have a high overall accuracy of 95%, while the polynomial kernel only had around 34% accuracy, so it seems as though the polynomial kernel may not be suitable for this particular dataset.    
   
However, from the confusion matrix obtained with sklearn in a previous attempt, we can see that most of the error in the polynomail kernel come from the label "2", meaning that it wrongly classified many of the images as "2". This shows that the parameters may not be optimised, causing such a huge error rate.

## C-SVC with optimised parameters

In this section we are asked to use grid search to find the parameters of the best performing model. From previous test, the best performing model was RBF, with an accuracy of 95.32%. In general, the RBF kernel is a reasonable first choice. This kernel nonlinearly maps samples into a higher dimensional space so it, unlike the linear kernel, can handle the case when the relation between class labels and attributes is nonlinear. The second reason is the number of hyperparameters which influences the complexity of model selection. The polynomial kernel has more hyperparameters than the RBF kernel.

There are two parameters for RBF kernel to be optimised: C and Gamma. The goal is to identify good parameters so that the classifier may accurately predict unknown (test) data, although it may not necessarily achieve the highest training accuracy.

We will be performing a grid search on C and Gamma using cross-validation. In v-fold cross-validation, we first divide the training set into v subsets of equal size. Sequentially one subset is tested using the classifier trained on the remaining v − 1 subsets. Thus, each instance of the whole training set is predicted once so the cross-validation accuracy is the percentage of data which are correctly classified. The cross-validation procedure can prevent the overfitting problem.

The grid-search is straightforward but seems naive. In fact, there are several advanced methods which can save computational cost by, for example, approximating the cross-validation rate. However, there are two motivations why we prefer the simple grid-search approach. One is that, psychologically, we may not feel safe to use methods which avoid doing an exhaustive parameter search by approximations or heuristics. The other reason is that the computational time required to find good parameters by gridsearch is not much more than that by advanced methods since there are only two parameters. 

Since doing a complete grid-search will be time-consuming, we will be using a coarse grid first using a randomly selected smaller subset of the dataset. After identifying a “better” region on the grid, a finer grid search on that region with the full dataset can be conducted. Libsvm's subset.py and grid.py will be heping us with that. However, in order to utilise them, we need to first convert our dataset to libsvm file format.

In [32]:
#convert file to libsvm format
x_train = pd.read_csv("X_train.csv",header=None).values
y_train = pd.read_csv("T_train.csv",header=None).values
list_x = []
list_y = []
for i, row in enumerate(x_train):
    _dict = {}
    for j, feature in enumerate(row):
        _dict[j + 1] = feature
    list_x.append(_dict)
for i, label in enumerate(y_train):
    list_y.append(label[0])
        
contain = []
for i, feature_dict in enumerate(list_x):
        _string = str(list_y[i]) + ' '
        for key, feature in feature_dict.items():
            _string = _string + str(key) + ':' + str(feature) + ' '
        _string += '\n'
        contain.append(_string)
with open("train.txt", 'w') as f:
        f.writelines(contain)

First we use subset.py to create a subset of the dataset of size 200. Parameter "s" == 1 means that we will be using random selection as opposed to stratified selection. The output file will be named subset.txt.

In [38]:
import os
import subprocess
args = ['python3', 'subset.py', '-s', '1', 'train.txt', str(200), 'subset.txt']
cmd = " ".join(args)
print(cmd)
subprocess.call(cmd, shell=True)

python3 subset.py -s 1 train.txt 200 subset.txt


0

Now we will use grid.py to do the coarse grid search on the subset of data. We will be using the default parameters, i.e.:
 * 5 fold cross-validation
 * the range of c (default -5,15,2), where the parameters begin,end,step -- c_range = 2^{begin,...,begin+k*step,...,end}
 * the range of g (default 3,-15,-2), where the parameters begin,end,step -- g_range = 2^{begin,...,begin+k*step,...,end}

In [51]:
args = ['python3', 'grid.py', 'subset.txt']
cmd = " ".join(args)
print(cmd)
subprocess.call(cmd, shell=True)

python3 grid.py subset.txt


0

In [49]:
print(open('subset.txt.out', 'r').readlines()[0])

log2c=5 log2g=-7 rate=95.5



We then perform once more grid search with the full dataset. This time, we will be using finer parameters around the ones we have found.

In [52]:
args = ['python3', 'grid.py', '-log2c','4.8,5.2,0.1','-log2g','-6.8,-7.2,-0.1', 'train.txt']
cmd = " ".join(args)
print(cmd)
subprocess.call(cmd, shell=True)

python3 grid.py -log2c 4.8,5.2,0.1 -log2g -6.8,-7.2,-0.1 train.txt


0

In [53]:
print(open('train.txt.out', 'r').readlines()[0])

log2c=4.999999999999999 log2g=-6.999999999999999 rate=98.04



The fine grid search still showed the same result, where C = 2^5 and G=2^-7, with a cross-validation rate of 98.04%. Let us now train the model with the given parameter and check the accuracy of the model on the testing data.

In [54]:
m = svm_train(train_label.flatten(), train_data, '-c 32 -g 0.0078125 -t 2')
rbf_result = svm_predict(test_label.flatten(),test_data,m)

Accuracy = 98.04% (2451/2500) (classification)


Indeed, the rbf model with the optimised parameter achieved higher classification accuracy as we expected.

## Combined kernel (unimplemented)

I am supposed to implement a pre-computed combined kernel using linear+rbf, with the parameter "-t 4". However, I was unable to do so.

As seen, this model has a higher precision and accuracy than all the other models aside from the optimised polynomial model. I believe it will be able to perform even better should we perform a grid search to optimise its parameters too, but that shall not be in the scope of this assignment.